Since the notebook server was updated, the following commands need to be run in a terminal before proceeding:
```
> pip install git+https://github.com/mtlam/PyPulse.git --upgrade --user
> pip install numdifftools
```
Also note: if you have existing pickled tim files in your working directory, remove them before running the notebook, otherwise you're likely to run into an issue with `alias_translation`. A warning has also been implemented to help diagnose this. Don't forget to restart the kernel before proceeding.

In [ ]:
from timing_analysis.lite_utils import *
from timing_analysis.par_checker import *
from timing_analysis.utils import *
from timing_analysis.dmx_utils import *
from timing_analysis.timingconfiguration import TimingConfiguration
from timing_analysis.outlier_utils import *
from timing_analysis.utils import apply_cut_flag, apply_cut_select
import yaml
from astropy import log
import pint.fitter
from pint.utils import dmxparse
import os

# For manual inspection:
from ipywidgets import *
from IPython.display import Image, display

In [ ]:
config = "configs/[psr_name].[nb or wb].yaml"  # fill in actual path
par_directory = None   # default location
tim_directory = None   # default location
tc = TimingConfiguration(config, par_directory=par_directory, tim_directory=tim_directory)
using_wideband = tc.get_toa_type() == 'WB'

# For manual inspection:
file_matches, toa_matches = tc.get_investigation_files()

In [ ]:
run_outlier_analysis = False
use_outlier_results = False
epochdrop_threads = 20

if run_outlier_analysis:
    mo,to = tc.get_model_and_toas(apply_initial_cuts=True)
    to = setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())

    # Run outlier analysis and assign outlier probabilities to TOAs (narrowband only)
    if not using_wideband:
        tc.check_outlier()
        calculate_pout(mo, to, tc)
        make_pout_cuts(mo, to, tc, outpct_threshold=8.0)

    epochalyptica(mo,to,tc,nproc=epochdrop_threads)

if use_outlier_results:        
    mo,to = tc.get_model_and_toas(excised=True)
if (not run_outlier_analysis) and (not use_outlier_results):
    mo,to = tc.get_model_and_toas(apply_initial_cuts=True)
    to = setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())

In [ ]:
# Quick breakdown of existing cut flags (automated excision)
cuts_dict = cut_summary(to,tc)

In [ ]:
# Apply manual cuts and check for redundancies
tc.manual_cuts(to)
to = setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())

In [ ]:
# Plot residuals vs. time after auto/manual cuts
from timing_analysis.plot_utils import plot_residuals_time
fo = tc.construct_fitter(to,mo)
plot_residuals_time(fo, restype='prefit')

In [ ]:
# Plot residuals & highlight manual cuts
highlight_cut_resids(to,mo,tc,ylim_good=True)

Running display_excise_dropdowns (as below) will cause dropdowns for each bad-file and bad-toa of interest to be displayed. You should choose whatever combinations of file and plot type you'd like to make, then run the cell after this one (containing the read_excise_dropdowns functions) to read in your choices. The optional arguments all_YFp, all_GTpd, and all_profile will default the checkboxes to those plot types. If not specified, the default will be 'None' (i.e. do not plot). Finally, the make_detective_plots function (next cell) will display your plots. Note that bad_toa entries with "profile" plots selected will plot profiles for all bad subints and bad subbands of interest (as well as the integrated profile). 

In [ ]:
# Display checkboxes
file_select_list, file_pav_list, toa_select_list, toa_pav_list = \
    display_excise_dropdowns(file_matches, toa_matches, all_YFp=False, all_GTpd=False, all_profile=False)

As described above, this block will read your choices from the above dropdowns and make plots. Bad files and bad toas are dealt with separately to allow for future use of the bad-toa subint/subband info.

In [ ]:
# Make plots (indicated by dropdown choices)
file_plot_list = read_excise_dropdowns(file_select_list, file_pav_list, file_matches)
toa_plot_list = read_excise_dropdowns(toa_select_list, toa_pav_list, toa_matches)
make_detective_plots(file_plot_list, file_matches)
make_detective_plots(toa_plot_list, toa_matches)

If you'd like to look at the cal file associated with any bad-file or bad-toa entry listed in the yaml, run the following two cells. Just as above, the first cell displays the checkboxes (which show the file name, not the cal name!). Then pick your plot types, and run the cell that follows to create the plots (in this case, there isn't a separate step to read the dropdowns).

In [ ]:
# Display cal file checkboxes
cal_select_list, full_cal_files = display_cal_dropdowns(file_matches, toa_matches)

In [ ]:
# Read dropdowns and make cal plots
read_plot_cal_dropdowns(cal_select_list, full_cal_files)

In [ ]:
# Print more detailed info about bad_toa entries
badtoas = tc.get_bad_toas()
if badtoas:
    for bt in badtoas:
        tc.badtoa_info(bt,to)

The following cells allow you to visualize auto-excised TOA files. Specify the cut flags you want with "cutkeys" (default is ['epochdrop', 'outlier10']), then pick what kind of plot you want to see with plot_type ('profile' [default], 'GTpd', or 'YFp').

In [ ]:
# Look at profiles for auto-excised TOAs (outlier10):
plot_list = display_auto_ex(tc, mo, cutkeys=['outlier10'], plot_type='profile')

In [ ]:
# Look at freq. vs. phase for auto-excised TOAs (epochdrop):
plot_list = display_auto_ex(tc, mo, cutkeys=['epochdrop'], plot_type='GTpd')